
# Titanic Analysis

This analysis will test the acuracy of a few deep learning models.  <br /> To open this notebook in Google Colab just click on the button below.<br /><a href="https://colab.research.google.com/github/istrategy/titanicdata/blob/master/notebooks/Titanic%20Deep%20Learning.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Import dependencies

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd

## Load whole dataset (it will be later splitted into train, test and validation)

In [2]:
df = pd.read_csv("https://www.48hours.ai/files/titanic/titanic-all.csv")
# displays data
df.head(n=2000)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [3]:
# describe data
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,0.377387,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.484918,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [4]:
# make a copy of the dataframe in case we need it later
titanic_unprocessed = df.copy()

## Prepare data

In [5]:
#dropping columns that will not be used for predictions
# process_df = df.drop(["PassengerId","Name","Ticket","Cabin"], axis=1)

# passenger id will not be dropped yet 
process_df = df.drop([ "Name","Ticket","Cabin"], axis=1)

#print number of collums with null values
print (process_df.isnull().sum() )
#processed_df


PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            263
SibSp            0
Parch            0
Fare             1
Embarked         2
dtype: int64


In [6]:
# As can be seen from the above table there are many null values.  
# We are going to fill null values with values

##identify mean ages
male_mean_age = process_df[process_df["Sex"]=="male"]["Age"].mean()
female_mean_age = process_df[process_df["Sex"]=="female"]["Age"].mean()
#print ("female mean age: %1.0f" %female_mean_age )
#print ("male mean age: %1.0f" %male_mean_age )
process_df.loc[ (process_df["Sex"]=="male") & (process_df["Age"].isnull()), "Age"] = male_mean_age
process_df.loc[ (process_df["Sex"]=="female") & (process_df["Age"].isnull()), "Age"] = female_mean_age

# fill fare with mean fare
mean_fare = process_df["Fare"].mean()

# cabin and embarked

process_df["Embarked"] = process_df["Embarked"].fillna("U") # U for unknown
process_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [7]:
# Check again for null values
print (process_df.head().isnull().sum() )
#process_df

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64


### Convert categorical variables to numeric values

If a variable contains male / female one hot encoding will encode it to 0 / 1

In [8]:
# the following categorical variables will be converted  sex & embarked

categorical_feature_mask = process_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = process_df.columns[categorical_feature_mask].tolist()

In [9]:
categorical_cols.append('Pclass') # manually add Pclass it is also a categorical variable
categorical_cols


['Sex', 'Embarked', 'Pclass']

In [10]:
process_df = pd.get_dummies(process_df, columns=categorical_cols, drop_first=True)

In [11]:
values_ds = process_df.iloc[:,2:]
values_ds

,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Embarked_U,Pclass_2,Pclass_3
0,22.000000,1,0,7.2500,1,0,1,0,0,1
1,38.000000,1,0,71.2833,0,0,0,0,0,0
2,26.000000,0,0,7.9250,0,0,1,0,0,1
3,35.000000,1,0,53.1000,0,0,1,0,0,0
4,35.000000,0,0,8.0500,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
1304,30.585228,0,0,8.0500,1,0,1,0,0,1
1305,39.000000,0,0,108.9000,0,0,0,0,0,0
1306,38.500000,0,0,7.2500,1,0,1,0,0,1
1307,30.585228,0,0,8.0500,1,0,1,0,0,1


In [12]:
target_ds  = process_df.iloc[:,1:2]
target_ds

,Survived
0,0
1,1
2,1
3,1
4,0
...,...
1304,0
1305,1
1306,0
1307,0


# Split data in train, test and validation batches

In [13]:
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(values_ds, target_ds, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

print(x_train, x_val, x_test)
print(y_train, y_val, y_test)

            Age  SibSp  Parch     Fare  Sex_male  Embarked_Q  Embarked_S  \
703   25.000000      0      0   7.7417         1           1           0   
724   27.000000      1      0  53.1000         1           0           1   
635   28.000000      0      0  13.0000         0           0           1   
893   62.000000      0      0   9.6875         1           1           0   
772   57.000000      0      0  10.5000         0           0           1   
...         ...    ...    ...      ...       ...         ...         ...   
923   33.000000      1      2  20.5750         0           0           1   
179   36.000000      0      0   0.0000         1           0           1   
1233  30.585228      1      9  69.5500         1           0           1   
830   15.000000      1      0  14.4542         0           0           0   
259   50.000000      0      1  26.0000         0           0           1   

      Embarked_U  Pclass_2  Pclass_3  
703            0         0         1  
724      

# Deep learning on data

## First model will be a shallow network

In [14]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from matplotlib import pyplot as plt

In [15]:
# in case you do not have keras installed uncomment the bottom line
# pip install keras
#print(x_train, x_val, x_test)
#print(y_train, y_val, y_test)

In [16]:
#Shallow network setup
model = Sequential()
model.add(Dense(5, activation='tanh', input_shape=(10,)))
model.add(Dense(20, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5)                 55        
_________________________________________________________________
dense_1 (Dense)              (None, 20)                120       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 196
Trainable params: 196
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='mean_squared_error', optimizer=SGD(lr=0.01), metrics=['accuracy'])

In [19]:
model.fit(x_train, y_train, batch_size=4, epochs=1000, verbose=1, validation_data=(x_val, y_val))

Epoch 1/1000
246/246 [==============================] - 1s 2ms/step - loss: 0.2271 - accuracy: 0.6259 - val_loss: 0.2294 - val_accuracy: 0.6173
Epoch 2/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2261 - accuracy: 0.6259 - val_loss: 0.2286 - val_accuracy: 0.6173
Epoch 3/1000
246/246 [==============================] - 0s 2ms/step - loss: 0.2248 - accuracy: 0.6259 - val_loss: 0.2271 - val_accuracy: 0.6173
Epoch 4/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2244 - accuracy: 0.6259 - val_loss: 0.2262 - val_accuracy: 0.6173
Epoch 5/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2235 - accuracy: 0.6259 - val_loss: 0.2251 - val_accuracy: 0.6173
Epoch 6/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2231 - accuracy: 0.6259 - val_loss: 0.2244 - val_accuracy: 0.6173
Epoch 7/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2225 - accuracy: 0.6259 - val_loss: 0.2249 - val_accuracy:

246/246 [==============================] - 0s 1ms/step - loss: 0.2180 - accuracy: 0.6595 - val_loss: 0.2215 - val_accuracy: 0.6429
Epoch 58/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2173 - accuracy: 0.6585 - val_loss: 0.2172 - val_accuracy: 0.6582
Epoch 59/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2180 - accuracy: 0.6595 - val_loss: 0.2196 - val_accuracy: 0.6633
Epoch 60/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2167 - accuracy: 0.6687 - val_loss: 0.2249 - val_accuracy: 0.6480
Epoch 61/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2186 - accuracy: 0.6656 - val_loss: 0.2241 - val_accuracy: 0.6480
Epoch 62/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2176 - accuracy: 0.6667 - val_loss: 0.2246 - val_accuracy: 0.6480
Epoch 63/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2180 - accuracy: 0.6595 - val_loss: 0.2228 - val_accuracy: 0.6429

246/246 [==============================] - 0s 1ms/step - loss: 0.2171 - accuracy: 0.6616 - val_loss: 0.2189 - val_accuracy: 0.6633
Epoch 114/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2178 - accuracy: 0.6514 - val_loss: 0.2199 - val_accuracy: 0.6531
Epoch 115/1000
246/246 [==============================] - 0s 2ms/step - loss: 0.2172 - accuracy: 0.6646 - val_loss: 0.2161 - val_accuracy: 0.6582
Epoch 116/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2183 - accuracy: 0.6544 - val_loss: 0.2194 - val_accuracy: 0.6633
Epoch 117/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2175 - accuracy: 0.6595 - val_loss: 0.2188 - val_accuracy: 0.6582
Epoch 118/1000
246/246 [==============================] - 0s 2ms/step - loss: 0.2179 - accuracy: 0.6585 - val_loss: 0.2192 - val_accuracy: 0.6633
Epoch 119/1000
246/246 [==============================] - 0s 2ms/step - loss: 0.2174 - accuracy: 0.6575 - val_loss: 0.2197 - val_accuracy: 

246/246 [==============================] - 0s 2ms/step - loss: 0.2167 - accuracy: 0.6626 - val_loss: 0.2200 - val_accuracy: 0.6531
Epoch 170/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2165 - accuracy: 0.6544 - val_loss: 0.2206 - val_accuracy: 0.6480
Epoch 171/1000
246/246 [==============================] - 0s 2ms/step - loss: 0.2165 - accuracy: 0.6595 - val_loss: 0.2236 - val_accuracy: 0.6480
Epoch 172/1000
246/246 [==============================] - 0s 2ms/step - loss: 0.2172 - accuracy: 0.6636 - val_loss: 0.2181 - val_accuracy: 0.6633
Epoch 173/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2173 - accuracy: 0.6616 - val_loss: 0.2191 - val_accuracy: 0.6633
Epoch 174/1000
246/246 [==============================] - 0s 2ms/step - loss: 0.2176 - accuracy: 0.6524 - val_loss: 0.2206 - val_accuracy: 0.6429
Epoch 175/1000
246/246 [==============================] - 0s 2ms/step - loss: 0.2173 - accuracy: 0.6575 - val_loss: 0.2201 - val_accuracy: 

246/246 [==============================] - 0s 1ms/step - loss: 0.2078 - accuracy: 0.6381 - val_loss: 0.2132 - val_accuracy: 0.6531
Epoch 226/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2057 - accuracy: 0.6483 - val_loss: 0.2033 - val_accuracy: 0.6173
Epoch 227/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2035 - accuracy: 0.6677 - val_loss: 0.1961 - val_accuracy: 0.7092
Epoch 228/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.2021 - accuracy: 0.6758 - val_loss: 0.1924 - val_accuracy: 0.6990
Epoch 229/1000
246/246 [==============================] - 0s 2ms/step - loss: 0.1994 - accuracy: 0.6809 - val_loss: 0.1969 - val_accuracy: 0.6837
Epoch 230/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1976 - accuracy: 0.6799 - val_loss: 0.2003 - val_accuracy: 0.6735
Epoch 231/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1999 - accuracy: 0.6728 - val_loss: 0.1879 - val_accuracy: 

...

Epoch 729/1000
246/246 [==============================] - 0s 959us/step - loss: 0.1398 - accuracy: 0.8308 - val_loss: 0.1478 - val_accuracy: 0.8061
Epoch 730/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1682 - accuracy: 0.7839 - val_loss: 0.1128 - val_accuracy: 0.8724
Epoch 731/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1322 - accuracy: 0.8389 - val_loss: 0.1118 - val_accuracy: 0.8673
Epoch 732/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1499 - accuracy: 0.8094 - val_loss: 0.1136 - val_accuracy: 0.8724
Epoch 733/1000
246/246 [==============================] - 0s 963us/step - loss: 0.1358 - accuracy: 0.8328 - val_loss: 0.1161 - val_accuracy: 0.8622
Epoch 734/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1312 - accuracy: 0.8440 - val_loss: 0.1133 - val_accuracy: 0.8724
Epoch 735/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1286 - accuracy: 0.8461 - val_loss: 0.11

Epoch 785/1000
246/246 [==============================] - 0s 980us/step - loss: 0.1494 - accuracy: 0.8114 - val_loss: 0.1324 - val_accuracy: 0.8316
Epoch 786/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1448 - accuracy: 0.8124 - val_loss: 0.1317 - val_accuracy: 0.8367
Epoch 787/1000
246/246 [==============================] - 0s 963us/step - loss: 0.1321 - accuracy: 0.8410 - val_loss: 0.1125 - val_accuracy: 0.8776
Epoch 788/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1404 - accuracy: 0.8308 - val_loss: 0.1139 - val_accuracy: 0.8673
Epoch 789/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1510 - accuracy: 0.8063 - val_loss: 0.1135 - val_accuracy: 0.8622
Epoch 790/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1540 - accuracy: 0.8084 - val_loss: 0.1167 - val_accuracy: 0.8673
Epoch 791/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1275 - accuracy: 0.8491 - val_loss: 0.20

Epoch 841/1000
246/246 [==============================] - 0s 972us/step - loss: 0.1521 - accuracy: 0.8012 - val_loss: 0.1160 - val_accuracy: 0.8622
Epoch 842/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1404 - accuracy: 0.8206 - val_loss: 0.1100 - val_accuracy: 0.8776
Epoch 843/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1514 - accuracy: 0.8094 - val_loss: 0.2151 - val_accuracy: 0.7194
Epoch 844/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1692 - accuracy: 0.7737 - val_loss: 0.1474 - val_accuracy: 0.8061
Epoch 845/1000
246/246 [==============================] - 0s 984us/step - loss: 0.1380 - accuracy: 0.8287 - val_loss: 0.1159 - val_accuracy: 0.8724
Epoch 846/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1484 - accuracy: 0.8165 - val_loss: 0.2303 - val_accuracy: 0.6582
Epoch 847/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1683 - accuracy: 0.7635 - val_loss: 0.16

246/246 [==============================] - 0s 1ms/step - loss: 0.1406 - accuracy: 0.8247 - val_loss: 0.1236 - val_accuracy: 0.8520
Epoch 898/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1379 - accuracy: 0.8338 - val_loss: 0.1101 - val_accuracy: 0.8673
Epoch 899/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1354 - accuracy: 0.8359 - val_loss: 0.1093 - val_accuracy: 0.8776
Epoch 900/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1259 - accuracy: 0.8502 - val_loss: 0.1114 - val_accuracy: 0.8776
Epoch 901/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1287 - accuracy: 0.8451 - val_loss: 0.1355 - val_accuracy: 0.8316
Epoch 902/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1417 - accuracy: 0.8277 - val_loss: 0.1564 - val_accuracy: 0.7908
Epoch 903/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1460 - accuracy: 0.8145 - val_loss: 0.1650 - val_accuracy: 

246/246 [==============================] - 0s 1ms/step - loss: 0.1418 - accuracy: 0.8236 - val_loss: 0.1136 - val_accuracy: 0.8673
Epoch 954/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1330 - accuracy: 0.8379 - val_loss: 0.1553 - val_accuracy: 0.7908
Epoch 955/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1480 - accuracy: 0.8175 - val_loss: 0.1124 - val_accuracy: 0.8724
Epoch 956/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1496 - accuracy: 0.8135 - val_loss: 0.1646 - val_accuracy: 0.7959
Epoch 957/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1472 - accuracy: 0.8124 - val_loss: 0.1994 - val_accuracy: 0.7296
Epoch 958/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1332 - accuracy: 0.8379 - val_loss: 0.1124 - val_accuracy: 0.8724
Epoch 959/1000
246/246 [==============================] - 0s 1ms/step - loss: 0.1160 - accuracy: 0.8675 - val_loss: 0.1110 - val_accuracy: 

## Conclusion

As we can see from the above epochs. There is no improvement in accuracy or val_accuracy during the epochs.<br> <br> Perhaps a deep network is not the right tool for the Titanic data set. Deep networks work better if there are many parameters with lots of data to work with. On the 48<font style="color:#d99116;">hours</font>.ai website we've  compiled a report with different prediction models. <br><br>The report can be downloaded <a href="https://www.48hours.ai/files/Titanic_Report.pdf">here https://www.48hours.ai/files/Titanic_Report.pdf</a>.  

### Additional notes

After changing the epochs to 1000 there seems to be some improvement in the learning. Some more investigation is needed.

## TO DO: Validate model against validation data